In [39]:
from flask import Flask,jsonify, request
from flask_cors import CORS, cross_origin

In [1]:
import pickle
import pandas as pd
import numpy as np

In [90]:
currency_rates={'AED': 0.27,
 'SAR': 0.27,
 'BHD': 2.65,
 'KWD': 3.25,
 'QAR': 0.27,
 'EUR': 1.0981,
 'CHF': 1.1191,
 'CZK': 0.047,
 'GBP': 1.2426,
 'RON': 0.2222,
 'USD': 1.0,
 'PLN': 0.2372,
 'HU1': 0.003,
 'SEK': 0.097,
 'CAD': 0.7484,
 'HKD': 0.1274,
 'AUD': 0.6714,
 'MYR': 0.2261,
 'SGD': 0.7514,
 'NZD': 0.6198}

In [2]:
with open('./model.pkl','rb') as f:
    model=pickle.load(f)

In [3]:
with open('./scalar_feature.pkl','rb') as f:
    feature_scalar=pickle.load(f)

In [4]:
with open('./scalar_target.pkl','rb') as f:
    result_scalar=pickle.load(f)

In [5]:
encoders=["dc","div","oc","pot","cca"]

In [6]:
label_encoders={}
for i in encoders:
    with open('./encoder'+i+'.pkl','rb') as f:
        label_encoders[i]=pickle.load(f)

In [7]:
selected_features=['SALES_ORG', 'DISTRIBUTION_CHANNEL', 'DIVISION',
       'RELEASED_CREDIT_VALUE', 'PURCHASE_ORDER_TYPE', 'COMPANY_CODE',
       'ORDER_CREATION_TIME', 'CREDIT_CONTROL_AREA', 'SOLD_TO_PARTY',
       'ORDER_CURRENCY', 'CREDIT_STATUS', 'UNIQUE_CUST_ID', 'OCDATE',
       'OCMONTH', 'OCDAY', 'RDDATE', 'RDMONTH', 'RDDAY', 'DAYS_TO_PROCESS']

In [8]:
label_encoded_columns=['DISTRIBUTION_CHANNEL', 'DIVISION', 'PURCHASE_ORDER_TYPE','CREDIT_CONTROL_AREA', 'ORDER_CURRENCY']

In [115]:
with open('./data.pkl','rb') as f:
    print(get_prediction(pickle.load(f)))

<class 'list'>
SL_NO                      0
CUSTOMER_ORDER_ID          0
SALES_ORG                  0
DISTRIBUTION_CHANNEL       0
DIVISION                   0
RELEASED_CREDIT_VALUE      0
PURCHASE_ORDER_TYPE        0
COMPANY_CODE               0
ORDER_CREATION_DATE        0
ORDER_CREATION_TIME        0
CREDIT_CONTROL_AREA        0
SOLD_TO_PARTY              0
ORDER_AMOUNT               0
REQUESTED_DELIVERY_DATE    0
ORDER_CURRENCY             0
CREDIT_STATUS              0
CUSTOMER_NUMBER            0
AMOUNT_IN_USD              0
UNIQUE_CUST_ID             0
id                         0
dtype: int64
[[4.0, 162.44232177734375, 178.3779135437012], [8.0, 705.1736450195312, 774.3511795959473], [10.0, 1734.192626953125, 1904.3169236572267], [12.0, 705.1736450195312, 774.3511795959473], [14.0, 705.1736450195312, 774.3511795959473], [15.0, 705.1736450195312, 774.3511795959473], [42.0, 478.9201965332031, 525.9022678131104], [43.0, 477.6278076171875, 524.4830955444336], [45.0, 904.852722167968

In [89]:
def convert_currency(a,currency):
    return currency_rates[currency]*float(a)

In [118]:
df={}
def get_prediction(df):
    data=pd.DataFrame(df)
    ids=list(map(int,data['id']))
    currency=data['ORDER_CURRENCY']
    
    data['REQUESTED_DELIVERY_DATE']=pd.to_datetime(data['REQUESTED_DELIVERY_DATE'],format='%d-%m-%Y')
    data['ORDER_CREATION_DATE']=pd.to_datetime(data['ORDER_CREATION_DATE'],format='%d-%m-%Y')
    
    data['OCDATE']=data['ORDER_CREATION_DATE'].dt.day
    data['OCMONTH']=data['ORDER_CREATION_DATE'].dt.month
    data['OCDAY']=data['ORDER_CREATION_DATE'].dt.weekday

    data['RDDATE']=data['REQUESTED_DELIVERY_DATE'].dt.day
    data['RDMONTH']=data['REQUESTED_DELIVERY_DATE'].dt.month
    data['RDDAY']=data['REQUESTED_DELIVERY_DATE'].dt.weekday

    data['DAYS_TO_PROCESS']=(data['REQUESTED_DELIVERY_DATE']-data['ORDER_CREATION_DATE']).dt.days
    data=data[selected_features]
    
    data['CREDIT_STATUS'].replace('','74',inplace=True)
    data['CREDIT_STATUS']=data['CREDIT_STATUS'].astype('float64')
    data['PURCHASE_ORDER_TYPE'].fillna('NEWCAT',inplace=True)
    
    data['DISTRIBUTION_CHANNEL']=list(label_encoders['dc'].transform(data['DISTRIBUTION_CHANNEL']))
    data['DIVISION']=list(label_encoders['div'].transform(data['DIVISION']))
    data['PURCHASE_ORDER_TYPE']=list(label_encoders['pot'].transform(data['PURCHASE_ORDER_TYPE']))
    data['CREDIT_CONTROL_AREA']=list(label_encoders['cca'].transform(data['CREDIT_CONTROL_AREA']))
    data['ORDER_CURRENCY']=list(label_encoders['oc'].transform(data['ORDER_CURRENCY']))
    
    
    data['RELEASED_CREDIT_VALUE']=data['RELEASED_CREDIT_VALUE'].apply(lambda a:np.log(a+1))
    
    data['ORDER_CREATION_TIME']=data['ORDER_CREATION_TIME'].astype('int64')
    data=feature_scalar.transform(data)
    sacl=result_scalar.inverse_transform(model.predict(data).reshape(-1,1))
    
    
    
    resp=list(pd.DataFrame(sacl).apply(np.exp).values.flatten())
    amt_usd=list(map(convert_currency,resp,currency))
    
    place_holder=list(np.array([[*ids],[*resp],[*amt_usd]]).transpose())
    response=[]
    for i in place_holder:
        response.append(list(i))
        
    return response
    
    
    
    

In [ ]:
app = Flask(__name__)
cors=CORS(app, origins='*')

@app.route('/predict', methods=["POST"])
def predict():
    resp=get_prediction(request.get_json())
    

    return jsonify({"data":(resp)})


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jun/2023 22:49:30] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 22:49:30] "POST /predict HTTP/1.1" 200 -


[{'SL_NO': 4, 'CUSTOMER_ORDER_ID': 930253442, 'SALES_ORG': 2381, 'DISTRIBUTION_CHANNEL': 'Greece', 'DIVISION': 'South-Region', 'RELEASED_CREDIT_VALUE': 0, 'PURCHASE_ORDER_TYPE': 'N000', 'COMPANY_CODE': 3290, 'ORDER_CREATION_DATE': '01-01-2022', 'ORDER_CREATION_TIME': '160020', 'CREDIT_CONTROL_AREA': 'SR01', 'SOLD_TO_PARTY': 947942786, 'ORDER_AMOUNT': 0, 'REQUESTED_DELIVERY_DATE': '06-01-2022', 'ORDER_CURRENCY': 'EUR', 'CREDIT_STATUS': '64', 'CUSTOMER_NUMBER': 1210351400, 'AMOUNT_IN_USD': 0, 'UNIQUE_CUST_ID': 12103514003290, 'id': 4}]
<class 'list'>
SL_NO                      0
CUSTOMER_ORDER_ID          0
SALES_ORG                  0
DISTRIBUTION_CHANNEL       0
DIVISION                   0
RELEASED_CREDIT_VALUE      0
PURCHASE_ORDER_TYPE        0
COMPANY_CODE               0
ORDER_CREATION_DATE        0
ORDER_CREATION_TIME        0
CREDIT_CONTROL_AREA        0
SOLD_TO_PARTY              0
ORDER_AMOUNT               0
REQUESTED_DELIVERY_DATE    0
ORDER_CURRENCY             0
CREDIT_STA

127.0.0.1 - - [22/Jun/2023 22:50:15] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 22:50:15] "POST /predict HTTP/1.1" 200 -


[{'SL_NO': 4, 'CUSTOMER_ORDER_ID': 930253442, 'SALES_ORG': 2381, 'DISTRIBUTION_CHANNEL': 'Greece', 'DIVISION': 'South-Region', 'RELEASED_CREDIT_VALUE': 0, 'PURCHASE_ORDER_TYPE': 'N000', 'COMPANY_CODE': 3290, 'ORDER_CREATION_DATE': '01-01-2022', 'ORDER_CREATION_TIME': '160020', 'CREDIT_CONTROL_AREA': 'SR01', 'SOLD_TO_PARTY': 947942786, 'ORDER_AMOUNT': 0, 'REQUESTED_DELIVERY_DATE': '06-01-2022', 'ORDER_CURRENCY': 'EUR', 'CREDIT_STATUS': '64', 'CUSTOMER_NUMBER': 1210351400, 'AMOUNT_IN_USD': 0, 'UNIQUE_CUST_ID': 12103514003290, 'id': 4}]
<class 'list'>
SL_NO                      0
CUSTOMER_ORDER_ID          0
SALES_ORG                  0
DISTRIBUTION_CHANNEL       0
DIVISION                   0
RELEASED_CREDIT_VALUE      0
PURCHASE_ORDER_TYPE        0
COMPANY_CODE               0
ORDER_CREATION_DATE        0
ORDER_CREATION_TIME        0
CREDIT_CONTROL_AREA        0
SOLD_TO_PARTY              0
ORDER_AMOUNT               0
REQUESTED_DELIVERY_DATE    0
ORDER_CURRENCY             0
CREDIT_STA

...

[{'SL_NO': 207, 'CUSTOMER_ORDER_ID': 885865869, 'SALES_ORG': 2845, 'DISTRIBUTION_CHANNEL': 'Sao Tome and Principe', 'DIVISION': 'North-Region', 'RELEASED_CREDIT_VALUE': 297.02, 'PURCHASE_ORDER_TYPE': '9999', 'COMPANY_CODE': 59, 'ORDER_CREATION_DATE': '03-01-2022', 'ORDER_CREATION_TIME': '52034', 'CREDIT_CONTROL_AREA': 'NR01', 'SOLD_TO_PARTY': 909572567, 'ORDER_AMOUNT': 249.6, 'REQUESTED_DELIVERY_DATE': '05-01-2022', 'ORDER_CURRENCY': 'RON', 'CREDIT_STATUS': '93', 'CUSTOMER_NUMBER': 1210142642, 'AMOUNT_IN_USD': 57.2763979548532, 'UNIQUE_CUST_ID': 121014264259, 'id': 207}]
<class 'list'>
SL_NO                      0
CUSTOMER_ORDER_ID          0
SALES_ORG                  0
DISTRIBUTION_CHANNEL       0
DIVISION                   0
RELEASED_CREDIT_VALUE      0
PURCHASE_ORDER_TYPE        0
COMPANY_CODE               0
ORDER_CREATION_DATE        0
ORDER_CREATION_TIME        0
CREDIT_CONTROL_AREA        0
SOLD_TO_PARTY              0
ORDER_AMOUNT               0
REQUESTED_DELIVERY_DATE    0
O

127.0.0.1 - - [22/Jun/2023 23:13:19] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 23:13:19] "POST /predict HTTP/1.1" 200 -


[{'SL_NO': 206, 'CUSTOMER_ORDER_ID': 766851940, 'SALES_ORG': 2829, 'DISTRIBUTION_CHANNEL': 'Jamaica', 'DIVISION': 'North-Region', 'RELEASED_CREDIT_VALUE': 0, 'PURCHASE_ORDER_TYPE': '9999', 'COMPANY_CODE': 135, 'ORDER_CREATION_DATE': '03-01-2022', 'ORDER_CREATION_TIME': '51953', 'CREDIT_CONTROL_AREA': 'NR01', 'SOLD_TO_PARTY': 883383299, 'ORDER_AMOUNT': 3117.77, 'REQUESTED_DELIVERY_DATE': '05-01-2022', 'ORDER_CURRENCY': 'CZK', 'CREDIT_STATUS': '64', 'CUSTOMER_NUMBER': 1230007723, 'AMOUNT_IN_USD': 142.647587839471, 'UNIQUE_CUST_ID': 1230007723135, 'id': 206}, {'SL_NO': 207, 'CUSTOMER_ORDER_ID': 885865869, 'SALES_ORG': 2845, 'DISTRIBUTION_CHANNEL': 'Sao Tome and Principe', 'DIVISION': 'North-Region', 'RELEASED_CREDIT_VALUE': 297.02, 'PURCHASE_ORDER_TYPE': '9999', 'COMPANY_CODE': 59, 'ORDER_CREATION_DATE': '03-01-2022', 'ORDER_CREATION_TIME': '52034', 'CREDIT_CONTROL_AREA': 'NR01', 'SOLD_TO_PARTY': 909572567, 'ORDER_AMOUNT': '269.03', 'REQUESTED_DELIVERY_DATE': '05-01-2022', 'ORDER_CURRENCY

127.0.0.1 - - [22/Jun/2023 23:45:54] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2023 23:45:54] "POST /predict HTTP/1.1" 200 -


[{'SL_NO': 4, 'CUSTOMER_ORDER_ID': 930253442, 'SALES_ORG': 2381, 'DISTRIBUTION_CHANNEL': 'Greece', 'DIVISION': 'South-Region', 'RELEASED_CREDIT_VALUE': 0, 'PURCHASE_ORDER_TYPE': 'N000', 'COMPANY_CODE': 3290, 'ORDER_CREATION_DATE': '01-01-2022', 'ORDER_CREATION_TIME': '160020', 'CREDIT_CONTROL_AREA': 'SR01', 'SOLD_TO_PARTY': 947942786, 'ORDER_AMOUNT': 0, 'REQUESTED_DELIVERY_DATE': '06-01-2022', 'ORDER_CURRENCY': 'EUR', 'CREDIT_STATUS': '64', 'CUSTOMER_NUMBER': 1210351400, 'AMOUNT_IN_USD': 0, 'UNIQUE_CUST_ID': 12103514003290, 'id': 4}]
<class 'list'>
SL_NO                      0
CUSTOMER_ORDER_ID          0
SALES_ORG                  0
DISTRIBUTION_CHANNEL       0
DIVISION                   0
RELEASED_CREDIT_VALUE      0
PURCHASE_ORDER_TYPE        0
COMPANY_CODE               0
ORDER_CREATION_DATE        0
ORDER_CREATION_TIME        0
CREDIT_CONTROL_AREA        0
SOLD_TO_PARTY              0
ORDER_AMOUNT               0
REQUESTED_DELIVERY_DATE    0
ORDER_CURRENCY             0
CREDIT_STA